In [0]:
import requests
from datetime import datetime
import logging
import urllib3

import sailpoint
import sailpoint.v2024
from sailpoint.configuration import (Configuration, ConfigurationParams)
from sailpoint.paginator import Paginator

from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType, MapType, DateType, ArrayType, IntegerType

In [0]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create console handler and set level to info
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# Create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatter to ch
ch.setFormatter(formatter)

# Add ch to logger
logger.addHandler(ch)

In [0]:
tenant = 'devrel-ga-5668'
client_name = 'sailpoint-readall-client'
secret_name = 'sailpoint-readall-secret'

In [0]:
logger.info('Defining API credentials')
api_url = f'https://{tenant}.api.identitynow-demo.com'
token_url = f'https://{tenant}.api.identitynow-demo.com/oauth/token'
client_id = dbutils.secrets.get(scope='sailpoint-devrel', key=client_name)
client_secret = dbutils.secrets.get(scope='sailpoint-devrel', key=secret_name)

In [0]:
logger.info('Configuring SailPoint SDK')
configurationParams = ConfigurationParams()
configurationParams.base_url = api_url
configurationParams.token_url = token_url
configurationParams.client_id = client_id
configurationParams.client_secret = client_secret
configuration = Configuration(configurationParams)
configuration.experimental = True
configuration.retries = urllib3.Retry(
    total=5,  # Total number of retries
    backoff_factor=10,  # Backoff factor for retry delay
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP status codes to retry on
    allowed_methods=["GET"]  # HTTP methods to retry
)

In [0]:
workflow_schema = StructType([
    StructField("name", StringType(), True),
    StructField("owner", MapType(StringType(), StringType(), True), True),
    StructField("description", StringType(), True),
    StructField("start", StringType(), True),
    StructField("steps", MapType(StringType(), StringType(), True), True),
    StructField("enabled", BooleanType(), True),
    StructField("trigger_type", StringType(), True),
    StructField("trigger_attributes", MapType(StringType(), StringType(), True), True),
    StructField("id", StringType(), True),
    StructField("execution_count", IntegerType(), True),
    StructField("failure_count", IntegerType(), True),
    StructField("created", TimestampType(), True),
    StructField("modified", TimestampType(), True),
    StructField("modified_by", MapType(StringType(), StringType(), True), True),
    StructField("creator", MapType(StringType(), StringType(), True), True)
])

In [0]:
async def get_all_workflows():
    all_workflows = []
    with sailpoint.v2024.ApiClient(configuration) as api_client:
        api_instance = sailpoint.v2024.WorkflowsApi(api_client)
        try:
            workflows = api_instance.list_workflows()
            for object in workflows:
                wflow = object.to_dict()
                wflow['owner'] = object.owner.to_dict()
                wflow['start'] = object.definition.start
                wflow['steps'] = object.definition.steps
                if object.trigger is not None:
                    wflow['trigger_type'] = object.trigger.type
                    wflow['trigger_attributes'] = object.trigger.attributes.to_dict()
                if object.modified_by is not None:
                    wflow['modified_by'] = object.modified_by.to_dict()
                wflow['creator'] = object.creator.to_dict()
                all_workflows.append(wflow)
        except Exception as e:
            print("Exception when calling WorkflowsApi->list_workflows: %s\n" % e)
            
    return all_workflows


In [0]:
all_wf = await get_all_workflows()
logger.info(f'Total Retrieved Workflows: {len(all_wf)}')

In [0]:
try:
    logger.info('Creating data frame...')
    wflows_df = spark.createDataFrame(all_wf, workflow_schema)
    today = datetime.utcnow().date()
    wflows_df = wflows_df.withColumn('day', lit(today))

    logger.info('Writing to delta...')
    wflows_df.write \
        .format('delta') \
        .mode('append') \
        .partitionBy('day') \
        .saveAsTable(f'`dunker_databricks_space`.sailpoint.workflows')

except Exception as e:
    logger.error(f'Error in creating data frame or writing to delta: {e}')